In [9]:
import catboost as cb
import gc
import joblib
import json
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import sys
sys.path.append("../")
import warnings
warnings.simplefilter("ignore")
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score, fbeta_score, make_scorer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [2]:
from utils.common import (
    sigmoid, pad_column_name
)
from utils.constants import *
from utils.eval_helpers import (
    plot_roc_curves, plot_feature_importance, 
    amex_metric, get_final_metric_df, amex_metric_np, lgb_amex_metric
)
from utils.eda_helpers import (
    plot_missing_proportion_barchart, 
    get_cols
)
from utils.extraction_helpers import read_file
from utils.feature_group import (
    CATEGORY_COLUMNS, CONTINUOUS_COLUMNS, NON_FEATURE_COLUMNS
)
from utils.feature_engineering_helpers import feature_gen_pipeline

In [3]:
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
from cycler import cycler
from IPython.display import display
from colorama import Fore, Back, Style
plt.rcParams['axes.facecolor'] = '#0057b8' # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])
plt.rcParams['text.color'] = 'w'

In [4]:
%load_ext autoreload
%autoreload

### Read Data

In [5]:
%%time
risk_df = read_file(f"{INTERIM_DATA_PATH}/v6/train_parquet/train_risk.parquet")
spend_payment_df = read_file(f"{INTERIM_DATA_PATH}/v6/train_parquet/train_spend_payment.parquet")

Shape of data: (5531451, 31)
Shape of data: (5531451, 24)
CPU times: user 5.47 s, sys: 3.23 s, total: 8.7 s
Wall time: 3.95 s


In [6]:
labels = read_file(f"{RAW_DATA_PATH}/train_labels.csv")

Shape of data: (458913, 2)


In [7]:
%%time
train = pd.concat([
    spend_payment_df, 
    risk_df.loc[:, get_cols(risk_df, "R_")]
], axis=1)

CPU times: user 1.22 s, sys: 927 ms, total: 2.15 s
Wall time: 2.15 s


In [10]:
%%time
train_agg, keep_column = feature_gen_pipeline(train)

Done insertion
Average done
Minimum done
Maximum done
Standard Deviation done
Last entry done
First entry done
Second last entry done
Third last entry done
MA2 for Recency 1 done
MA2 for Recency 2 done
MA2 for Recency 3 done
MA3 for Recency 1 done
MA3 for Recency 2 done
MA3 for least Recency done


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:24<00:00,  1.92it/s]


CPU times: user 47.3 s, sys: 10.2 s, total: 57.6 s
Wall time: 57.9 s


In [12]:
train_agg.head(3)

,P_2_last,S_3_last,P_3_last,S_5_last,S_6_last,S_7_last,S_8_last,S_12_last,S_13_last,S_15_last,...,R_5_previous_sprint,R_5_acceleration,R_5_range,R_5_displacement,R_5_displacement_ratio,R_5_velocity,R_5_last_minus_avg,R_5_last_minus_midpoint,R_5_coef_var,R_5_trend_index
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.93,0.13,0.62,0.03,0,0.1,0.48,0.18,8.0,0.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.88,0.16,0.57,0.04,0,0.2,0.40,0.19,2.0,0.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.88,NaN,0.62,0.00,1,NaN,NaN,0.19,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# len(features_to_drop)
# train_agg = train_agg.drop(columns=list(features_to_drop), errors="ignore")

In [14]:
target = labels["target"].values

In [15]:
%%time
train_agg = train_agg.drop(columns=NON_FEATURE_COLUMNS + ["target"], errors="ignore")
gc.collect()

CPU times: user 5.09 s, sys: 11.2 s, total: 16.3 s
Wall time: 21.6 s


0

In [16]:
cat_features = train_agg.select_dtypes("category").columns.tolist()

In [17]:
train_agg.loc[:, cat_features] = train_agg.loc[:, cat_features].astype(float).fillna(-1).astype(int).astype("category")

In [18]:
train_agg.shape, target.shape

((458913, 1364), (458913,))

### Train Catboost

In [20]:
param = {
    "objective": "Logloss",
    "colsample_bylevel": 0.05,
    "depth": 6,
    "boosting_type": "Ordered",
    "bootstrap_type": "MVS",
    "n_estimators": 1000,
    "l2_leaf_reg": 3,
    "min_data_in_leaf": 1024,
    "learning_rate": 0.05,
    "max_bin": 127,
    "used_ram_limit": "16gb",
}

In [21]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1020)

In [ ]:
train_score_list, val_score_list = [], []
for fold, (idx_tr, idx_va) in zip(range(1, 5+1), kf.split(train_agg, target)):
    fold = str(fold)
    X_train, y_train = train_agg.iloc[idx_tr], target[idx_tr]
    X_val, y_val = train_agg.iloc[idx_va], target[idx_va]
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning)
        model = cb.CatBoostClassifier(**param)
        model.fit(
            X_train, 
            y_train, 
            eval_set=[(X_val, y_val)], 
            verbose=100, 
            cat_features=cat_features,
            early_stopping_rounds=100
        )
    y_train_pred = model.predict(X_train, prediction_type='RawFormulaVal')
    train_score, train_g, train_t4 = amex_metric(y_train, y_train_pred)
    train_data, X_train, y_train = None, None, None
    y_val_pred = model.predict(X_val, prediction_type='RawFormulaVal')
    val_score, val_g, val_t4 = amex_metric(y_val, y_val_pred)
    valid_data, X_val, y_val = None, None, None
    train_score_list.append(train_score)
    val_score_list.append(val_score)
    print(f"{Fore.BLUE}{Style.BRIGHT}Fold {fold} | Train Score = {train_score:.5f} ({train_g:.4f}, {train_t4:.4f})")
    print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | Val Score = {val_score:.5f} ({val_g:.4f}, {val_t4:.4f}){Style.RESET_ALL}")
    print(f"Clear cache {gc.collect()}")

0:	learn: 0.6404905	test: 0.6407026	best: 0.6407026 (0)	total: 237ms	remaining: 3m 57s
100:	learn: 0.2659042	test: 0.2709228	best: 0.2709228 (100)	total: 14.7s	remaining: 2m 10s
200:	learn: 0.2595988	test: 0.2657067	best: 0.2657067 (200)	total: 29.4s	remaining: 1m 57s
300:	learn: 0.2560411	test: 0.2630463	best: 0.2630463 (300)	total: 45s	remaining: 1m 44s
400:	learn: 0.2533009	test: 0.2614200	best: 0.2614200 (400)	total: 59.3s	remaining: 1m 28s
500:	learn: 0.2511526	test: 0.2604382	best: 0.2604382 (500)	total: 1m 15s	remaining: 1m 15s
600:	learn: 0.2493851	test: 0.2597818	best: 0.2597818 (600)	total: 1m 31s	remaining: 1m
700:	learn: 0.2478715	test: 0.2593069	best: 0.2593069 (700)	total: 1m 47s	remaining: 45.7s
800:	learn: 0.2463750	test: 0.2589566	best: 0.2589540 (799)	total: 2m 6s	remaining: 31.3s
900:	learn: 0.2450116	test: 0.2586375	best: 0.2586375 (900)	total: 2m 24s	remaining: 15.9s
999:	learn: 0.2436309	test: 0.2583618	best: 0.2583618 (999)	total: 2m 41s	remaining: 0us

bestTest 

### Tune Catboost using Optuna (KFold)

In [53]:
def objective(trial):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1020)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.03, 0.1, log=True),
        "depth": trial.suggest_int("depth", 4, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bernoulli", "MVS"]
        ),
        "n_estimators": trial.suggest_categorical("n_estimators", [1000, 1200, 1400]),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10, log=True),
        "min_data_in_leaf": trial.suggest_categorical("min_data_in_leaf", [512, 1024, 2048]),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.025, 0.05, 0.1]),
        "max_bin": trial.suggest_categorical("max_bin", [63, 127, 255]),
        "used_ram_limit": "12gb",
    }

    if param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.6, 0.85)
    
    print(param)
    train_score_list, val_score_list = [], []
    for fold, (idx_tr, idx_va) in zip(range(1, 5+1), kf.split(train_agg, target)):
        fold = str(fold)
        X_train, y_train = train_agg.iloc[idx_tr], target[idx_tr]
        X_val, y_val = train_agg.iloc[idx_va], target[idx_va]
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=UserWarning)
            model = cb.CatBoostClassifier(**param)
            model.fit(
                X_train, 
                y_train, 
                eval_set=[(X_val, y_val)], 
                verbose=0, 
                cat_features=cat_features,
                early_stopping_rounds=100
            )
        y_train_pred = model.predict(X_train, prediction_type='RawFormulaVal')
        train_score, train_g, train_t4 = amex_metric(y_train, y_train_pred)
        train_data, X_train, y_train = None, None, None
        y_val_pred = model.predict(X_val, prediction_type='RawFormulaVal')
        val_score, val_g, val_t4 = amex_metric(y_val, y_val_pred)
        valid_data, X_val, y_val = None, None, None
        train_score_list.append(train_score)
        val_score_list.append(val_score)
        if val_score > best_scores_json["validation"][fold]:
            best_scores_json["train"][fold] = train_score
            best_scores_json["validation"][fold] = val_score
            with open(f'{CURRENT_EXP_PATH}/best_scores.json', "w") as outfile:
                json.dump(best_scores_json, outfile)
            joblib.dump(model, f'{CURRENT_EXP_PATH}/models/model{fold}.pkl')
        elif np.mean(train_score_list) >= np.mean(list(best_scores_json["train"].values())) + 0.02:
            print(f"Train score too high (overfitting), start a new trial")
            return np.mean(val_score_list)
        print(f"{Fore.BLUE}{Style.BRIGHT}Fold {fold} | Train Score = {train_score:.5f} ({train_g:.4f}, {train_t4:.4f})")
        print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | Val Score = {val_score:.5f} ({val_g:.4f}, {val_t4:.4f}){Style.RESET_ALL}")
        print(f"Clear cache {gc.collect()}")
        
    return np.mean(val_score_list)

In [54]:
# study = joblib.load(f"{CURRENT_EXP_PATH}/optuna_study.pkl")

In [57]:
# study = optuna.create_study(direction="maximize")

In [59]:
study.optimize(objective, n_trials=50)

{'objective': 'Logloss', 'colsample_bylevel': 0.06079765732099717, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1400, 'l2_leaf_reg': 7.6044718629016215, 'min_data_in_leaf': 2048, 'learning_rate': 0.07, 'max_bin': 63, 'used_ram_limit': '12gb'}
Fold 1 | Train Score = 0.84184 (0.9419, 0.7417)
Fold 1 | Val Score = 0.79243 (0.9219, 0.6629)
Clear cache 0
Fold 2 | Train Score = 0.84106 (0.9414, 0.7407)
Fold 2 | Val Score = 0.79242 (0.9233, 0.6616)
Clear cache 0
Fold 3 | Train Score = 0.84232 (0.9417, 0.7429)
Fold 3 | Val Score = 0.79439 (0.9242, 0.6646)
Clear cache 0
Fold 4 | Train Score = 0.84181 (0.9416, 0.7420)
Fold 4 | Val Score = 0.79802 (0.9246, 0.6714)
Clear cache 0


[I 2022-07-30 10:29:18,789] Trial 23 finished with value: 0.7944563728274476 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.06079765732099717, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1400, 'l2_leaf_reg': 7.6044718629016215, 'min_data_in_leaf': 2048, 'learning_rate': 0.07, 'max_bin': 63}. Best is trial 17 with value: 0.7952103875641641.


Fold 5 | Train Score = 0.84108 (0.9411, 0.7411)
Fold 5 | Val Score = 0.79502 (0.9254, 0.6647)
Clear cache 0
{'objective': 'Logloss', 'colsample_bylevel': 0.06678481135488246, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1400, 'l2_leaf_reg': 5.523346450477207, 'min_data_in_leaf': 2048, 'learning_rate': 0.07, 'max_bin': 63, 'used_ram_limit': '12gb'}
Fold 1 | Train Score = 0.85918 (0.9482, 0.7702)
Fold 1 | Val Score = 0.79167 (0.9219, 0.6615)
Clear cache 0
Fold 2 | Train Score = 0.85973 (0.9483, 0.7712)
Fold 2 | Val Score = 0.79354 (0.9238, 0.6633)
Clear cache 0
Fold 3 | Train Score = 0.85960 (0.9480, 0.7712)
Fold 3 | Val Score = 0.79511 (0.9243, 0.6659)
Clear cache 0
Fold 4 | Train Score = 0.86155 (0.9486, 0.7745)
Fold 4 | Val Score = 0.79594 (0.9243, 0.6675)
Clear cache 0


[I 2022-07-30 11:00:49,370] Trial 24 finished with value: 0.794272749764656 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.06678481135488246, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1400, 'l2_leaf_reg': 5.523346450477207, 'min_data_in_leaf': 2048, 'learning_rate': 0.07, 'max_bin': 63}. Best is trial 17 with value: 0.7952103875641641.


Fold 5 | Train Score = 0.85856 (0.9474, 0.7697)
Fold 5 | Val Score = 0.79510 (0.9253, 0.6649)
Clear cache 0
{'objective': 'Logloss', 'colsample_bylevel': 0.06047710022425621, 'depth': 5, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1400, 'l2_leaf_reg': 3.2517951534674845, 'min_data_in_leaf': 2048, 'learning_rate': 0.07, 'max_bin': 63, 'used_ram_limit': '12gb'}
Fold 1 | Train Score = 0.82766 (0.9366, 0.7187)
Fold 1 | Val Score = 0.79261 (0.9216, 0.6636)
Clear cache 0
Fold 2 | Train Score = 0.82760 (0.9364, 0.7188)
Fold 2 | Val Score = 0.79267 (0.9232, 0.6621)
Clear cache 0
Fold 3 | Train Score = 0.82670 (0.9361, 0.7173)
Fold 3 | Val Score = 0.79450 (0.9240, 0.6650)
Clear cache 0
Fold 4 | Train Score = 0.82698 (0.9362, 0.7178)
Fold 4 | Val Score = 0.79628 (0.9243, 0.6683)
Clear cache 0


...

[I 2022-07-30 21:31:13,346] Trial 52 finished with value: 0.7934968062672549 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.042751802624111254, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1200, 'l2_leaf_reg': 2.074222982786614, 'min_data_in_leaf': 2048, 'learning_rate': 0.07, 'max_bin': 255}. Best is trial 48 with value: 0.7952443121251143.


Fold 5 | Train Score = 0.84784 (0.9434, 0.7523)
Fold 5 | Val Score = 0.79445 (0.9250, 0.6639)
Clear cache 0
{'objective': 'Logloss', 'colsample_bylevel': 0.06424643294980259, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1400, 'l2_leaf_reg': 5.108272805150234, 'min_data_in_leaf': 2048, 'learning_rate': 0.07, 'max_bin': 63, 'used_ram_limit': '12gb'}
Fold 1 | Train Score = 0.84267 (0.9424, 0.7430)
Fold 1 | Val Score = 0.79171 (0.9218, 0.6617)
Clear cache 0
Fold 2 | Train Score = 0.84232 (0.9419, 0.7428)
Fold 2 | Val Score = 0.79305 (0.9233, 0.6628)
Clear cache 0
Fold 3 | Train Score = 0.84319 (0.9420, 0.7444)
Fold 3 | Val Score = 0.79578 (0.9244, 0.6672)
Clear cache 0
Fold 4 | Train Score = 0.84342 (0.9421, 0.7448)
Fold 4 | Val Score = 0.79694 (0.9243, 0.6696)
Clear cache 0


[I 2022-07-30 21:58:19,335] Trial 53 finished with value: 0.794641883578542 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.06424643294980259, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1400, 'l2_leaf_reg': 5.108272805150234, 'min_data_in_leaf': 2048, 'learning_rate': 0.07, 'max_bin': 63}. Best is trial 48 with value: 0.7952443121251143.


Fold 5 | Train Score = 0.84165 (0.9415, 0.7418)
Fold 5 | Val Score = 0.79574 (0.9253, 0.6662)
Clear cache 0
{'objective': 'Logloss', 'colsample_bylevel': 0.06809959507321718, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1400, 'l2_leaf_reg': 4.168814994770979, 'min_data_in_leaf': 2048, 'learning_rate': 0.07, 'max_bin': 63, 'used_ram_limit': '12gb'}
Fold 1 | Train Score = 0.86072 (0.9485, 0.7729)
Fold 1 | Val Score = 0.79144 (0.9216, 0.6612)
Clear cache 0
Fold 2 | Train Score = 0.85640 (0.9470, 0.7658)
Fold 2 | Val Score = 0.79209 (0.9234, 0.6608)
Clear cache 0
Fold 3 | Train Score = 0.85363 (0.9458, 0.7614)
Fold 3 | Val Score = 0.79385 (0.9240, 0.6637)
Clear cache 0



KeyboardInterrupt



In [60]:
study.best_trial

FrozenTrial(number=48, values=[0.7952443121251143], datetime_start=datetime.datetime(2022, 7, 30, 19, 47, 2, 641815), datetime_complete=datetime.datetime(2022, 7, 30, 20, 16, 32, 872054), params={'objective': 'Logloss', 'colsample_bylevel': 0.06413298980628422, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1400, 'l2_leaf_reg': 4.2290041321509095, 'min_data_in_leaf': 2048, 'learning_rate': 0.07, 'max_bin': 63}, distributions={'objective': CategoricalDistribution(choices=('Logloss',)), 'colsample_bylevel': LogUniformDistribution(high=0.1, low=0.03), 'depth': IntUniformDistribution(high=12, low=4, step=1), 'boosting_type': CategoricalDistribution(choices=('Ordered', 'Plain')), 'bootstrap_type': CategoricalDistribution(choices=('Bernoulli', 'MVS')), 'n_estimators': CategoricalDistribution(choices=(1000, 1200, 1400)), 'l2_leaf_reg': LogUniformDistribution(high=10.0, low=1.0), 'min_data_in_leaf': CategoricalDistribution(choices=(512, 1024, 2048)), 'learning

In [61]:
joblib.dump(study, f"{CURRENT_EXP_PATH}/optuna_study.pkl")

['../experiments/catboost_new_exp/optuna_study.pkl']

In [62]:
study_df = study.trials_dataframe()

In [11]:
# for col in get_cols(study_df, "params"):
#     sns.scatterplot(data=study_df, x=study_df[col], y=study_df["value"])
#     plt.show()